In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import sentence_transformers
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

c:\Users\rakes\anaconda3\envs\medical_chatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "7ad73200-8c0f-4d58-8dc5-0731ed6224ed"
PINECONE_API_ENV = "aped-4627-b74a"

In [4]:
#Extarct data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob = "*.pdf",
                    loader_cls = PyPDFLoader)
    documents = loader.load()
    return documents

In [5]:
extracted_data = load_pdf("data/")

In [6]:
extracted_data

[Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='Sugar (Blood Glucose) \nYoungsters (under 60 years):  \n\uf0b7 Fasting: 70-99 mg/dL \n\uf0b7 After Meal (Postprandial):  70-139 mg/dL \nElders (60 years and above):  \n\uf0b7 Fasting: 80-130 mg/dL \n\uf0b7 After Meal (Postprandial):  80-180 mg/dL \nImpacts:  \n\uf0b7 Low (Hypoglycemia):  Shaking, sweating, confusion, dizziness, and in severe cases, loss \nof consciousness. \n\uf0b7 High (Hyperglycemia):  Increased thirst, frequent urination, fatigue, blurred vision, and \nrisk of diabetes complications over time. \nPlatelets Count \nAll Ages:  \n\uf0b7 150,000 - 450,000 cells/µL  \nImpacts:  \n\uf0b7 Low (Thrombocytopenia):  Increased risk of bleeding and bruising. \n\uf0b7 High (Thrombocytosis):  Increased risk of blood clots, which can lead to stroke or heart \nattack. \nNeutrophils \nYoungsters:  \n\uf0b7 40-70% of total white blood cells  \nElders: \n\uf0b7 40-70% of total white blood cells  \nImpacts:

In [7]:
#create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=50, chunk_overlap = 10)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks = text_split(extracted_data)
print("Length of my chunk: ",len(text_chunks))

Length of my chunk:  98


In [9]:
text_chunks

[Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='Sugar (Blood Glucose)'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='Youngsters (under 60 years):'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='\uf0b7 Fasting: 70-99 mg/dL'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='\uf0b7 After Meal (Postprandial):  70-139 mg/dL'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='Elders (60 years and above):'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='\uf0b7 Fasting: 80-130 mg/dL'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='\uf0b7 After Meal (Postprandial):  80-180 mg/dL'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='Impacts:'),
 Document(metadata={'source': 'data\\Sample Data.pdf', 'page': 0}, page_content='\uf0b

In [10]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings() 

c:\Users\rakes\anaconda3\envs\medical_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\rakes\anaconda3\envs\medical_chatbot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
query_result = embeddings.embed_query("Hello World")
print("Length", len(query_result))

Length 384


In [14]:
query_result

[-0.03447728976607323,
 0.03102322667837143,
 0.006735010538250208,
 0.026108987629413605,
 -0.03936200588941574,
 -0.16030244529247284,
 0.06692402064800262,
 -0.0064414627850055695,
 -0.047450508922338486,
 0.01475893147289753,
 0.07087531685829163,
 0.05552755668759346,
 0.01919335313141346,
 -0.026251355186104774,
 -0.010109529830515385,
 -0.026940520852804184,
 0.02230735309422016,
 -0.0222266037017107,
 -0.1496925950050354,
 -0.01749310828745365,
 0.007676230277866125,
 0.05435226112604141,
 0.0032544718123972416,
 0.031725943088531494,
 -0.08462142199277878,
 -0.02940600924193859,
 0.05159568786621094,
 0.04812406748533249,
 -0.00331477215513587,
 -0.05827920883893967,
 0.041969284415245056,
 0.02221072092652321,
 0.128188818693161,
 -0.022338923066854477,
 -0.011656236834824085,
 0.06292837113142014,
 -0.03287626430392265,
 -0.09122605621814728,
 -0.03117534890770912,
 0.05269956588745117,
 0.04703482612967491,
 -0.0842030718922615,
 -0.030056139454245567,
 -0.02074483036994934

In [15]:
pc = Pinecone(api_key = PINECONE_API_KEY)
index_name = "medical-chatbot"
index = pc.Index(index_name)
model_name="sentence-transformers/all-MiniLM-L6-v2"
embeddings = SentenceTransformer(model_name)


embeddings_model = [embeddings.encode(text.page_content).tolist() for text in text_chunks]

# Create a list of IDs and metadata for the embeddings
vectors_with_metadata = [
    (f"id_{i}", embeddings_model[i], {"text": text_chunks[i].page_content})
    for i in range(len(embeddings_model))
]

# Upsert (upload and insert) the embeddings with metadata into the Pinecone index
index.upsert(vectors=vectors_with_metadata)


'\n# Create a list of IDs and metadata for the embeddings\nvectors_with_metadata = [\n    (f"id_{i}", embeddings_model[i], {"text": text_chunks[i].page_content})\n    for i in range(len(embeddings_model))\n]\n\n# Upsert (upload and insert) the embeddings with metadata into the Pinecone index\nindex.upsert(vectors=vectors_with_metadata)\n'

In [16]:
# Define a function to perform the query
def retrieve_from_pinecone(query_text, top_k=2):
    query_embedding = embeddings.encode(query_text)
    # Convert numpy array to list
    query_embedding_list = query_embedding.tolist()
    response = index.query(vector=query_embedding_list, top_k=top_k)
    return response['matches']
# Example usage
query_text = "What are the symptoms of sugar?"
results = retrieve_from_pinecone(query_text)
print(results)

[{'id': 'id_13', 'score': 0.623755634, 'values': []}, {'id': 'id_0', 'score': 0.565182269, 'values': []}]


In [17]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
For list of answers, give in bulletin.

For the given medical values, analyse it and give a short report on it. Give medical sugessions and food diet.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Initialize the language model
llm = CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                    model_type="llama",
                    config={'max_new_tokens': 512, 'temperature': 0.8})

def answer_question(question):
    matches = retrieve_from_pinecone(question)
    # Create a context based on IDs if metadata is not available
    context = "\n".join([f"ID: {match['id']}, Score: {match['score']}" for match in matches])
    prompt = PROMPT.format(context=context, question=question)
    return llm(prompt)

# Example usage
answer = answer_question("BLOOD SUGAR FASTING ‘ :210 mg/dl, BLOOD SUGAR pp ( 2 HRS ) : 383 mg/dl, GLYCOSYLATED HB ( HB AIC ) : 11.10 %")
print(answer)


c:\Users\rakes\anaconda3\envs\medical_chatbot\lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


For fasting blood sugar levels of 210 mg/dl, you are considered to have impaired fasting glucose. This means that your body is having difficulty regulating your blood sugar levels, which can increase your risk for developing type 2 diabetes and other health problems. To help manage your blood sugar levels, consider the following dietary changes:
* Eat regular meals to maintain stable blood sugar levels throughout the day. Aim for three main meals and one or two snacks in between.
* Choose complex carbohydrates such as whole grains, fruits, vegetables, legumes, and nuts. These foods are rich in fiber, which can help slow down digestion and absorption of carbohydrates, thus preventing spikes in blood sugar levels.
* Limit your intake of refined carbohydrates such as white bread, sugary drinks, and baked goods. These foods are low in fiber and can cause a rapid increase in blood sugar levels.
* Include protein and healthy fats in your meals to help regulate blood sugar levels and improve 